In [42]:
import sqlite3

# ## Open enhance_google_places.db
# with sqlite3.connect('result\enhanced_google_places.db') as conn:
#     cursor = conn.cursor()
    
#     cursor.execute("SELECT DISTINCT categories_0 from places")
#     categories = cursor.fetchall()
#     print(categories)

conn = sqlite3.connect('result\placev2.db')
cursor = conn.cursor()

## 4 columns
cursor.execute("SELECT DISTINCT categories_0, categoryname, cuisine_tags, primary_cuisine from places")
categories = cursor.fetchall()

cater_bags = set()
for a, b, c, d in categories:
    cater_list_as_string = str(a) + ',' + str(b) + ',' + str(c) + ',' + str(d)
    cater_list = cater_list_as_string.split(',')
    cater_list = [cater.strip() for cater in cater_list]
    for cater in cater_list:
        cater_bags.add(cater)

with open('result\cater_bags.txt', 'w', encoding='utf-8') as f:
    for a in cater_bags:
        f.write(a)
        f.write('\n')


In [43]:
print(cater_bags)
print(len(cater_bags))

{'Cake shop', 'Porridge', '', 'Halal', 'Takeout', 'Meeting planning service', 'Grocery store', 'Chinese noodle', 'Trung tâm văn hoá', 'Trung tâm đào tạo', 'Wine store', 'Brunch', 'Hawker stall', 'Chinese', 'Food court', 'Công ty xây dựng nhà gỗ', 'Dịch vụ cho thuê canô và xuồng caiac', 'Vegetarian restaurant', 'Khách sạn', 'Motorcycle parts store', 'Japanese', 'Chùa phật giáo', 'Restaurant', 'Dịch vụ giặt ủi', 'Dịch vụ sản xuất video', 'Chicken', 'British restaurant', 'Nhà máy cơ khí', 'Yakitori', 'Hamburger restaurant', 'Quán rượu', 'Quán bar', 'Nhà hàng Ấn Độ', 'Sushi', 'Biệt thự', 'Banquet hall', 'Nhà bán buôn mỹ phẩm', 'Soup restaurant', 'Asian', 'Đại lý du lịch bằng xe buýt', 'Dịch vụ sửa chữa máy in', 'Shopping mall', 'Seafood donburi restaurant', 'Seafood donburi', 'Coffee shop', 'Điểm mốc văn hóa', 'Seafood', 'Tour operator', 'Californian restaurant', 'Mobile caterer', 'Western', 'Hunan restaurant', 'Cửa hàng quần áo nữ', 'Nơi tổ chức sự kiện', 'Indian Muslim restaurant', 'Frie

In [ ]:
CATEGORIES = ["accommodation", "activity", "catering",
        "commercial", "entertainment",
        "heritage", "leisure",
        "manufacturing", "natural",
        "religion", "rental", 
        "service", "tourism"]


In [45]:
## format return: "cater : binarymask of length 13"
def binary_to_cater(binary_mask):
    return [CATEGORIES[i] for i in range(len(binary_mask)) if binary_mask[i] == '1']

def binary_string_to_num(binary_string):
    return int(binary_string, 2)


In [46]:
   
cater_to_bin = {}
with open('result/auto_categorize.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line == '':
            continue
        cater, bin_mask = line.split(':')
        cater, bin_mask = cater.strip(), bin_mask.strip()
        print(bin_mask)
        assert len(bin_mask) == 13
        cater_to_bin[cater] = bin_mask



0011000000000
0010000000000
0010000000000
0010000000000
0000000000010
0001000000000
0010000000000
0000110000000
0000000000010
0001000000000
0010000000000
0010000000000
0010000000000
0011000000000
0000000100010
0000001000100
0010000000000
1000000000010
0001000000000
0010000000000
0000010001000
0010000000000
0000000000010
0000000000010
0010000000000
0010000000000
0000000100000
0010000000000
0010000000000
0010001000000
0010101000000
0010000000000
0010000000000
1000000000000
0010100000010
0001000000000
0010000000000
0010000000000
0000000000111
0000000000010
0001100000000
0010000000000
0010000000000
0010001000000
0000010000001
0010000000000
0000000000011
0010000000000
0010000000010
0010000000000
0010000000000
0001000000000
0000100000010
0010000000000
0010000000000
0100101000000
0000000100000
0010000000000
0100001000010
0001000000000
0010000000000
0010000000000
0000001010000
0000100000000
0010101000000
0000110000000
0010000000000
0010000000000
0000000000011
0000000000011
0010000000000
000100

In [47]:
print(cater_to_bin.get('Công viên'))

0000001010000


In [48]:
def process_catergories(catergories):
    ## categories to binary mask
    mask_list = []
    for a, b, c, d in catergories:
        cater_list_as_string = str(a) + ',' + str(b) + ',' + str(c) + ',' + str(d)
        cater_list = cater_list_as_string.split(',')
        cater_list = [cater.strip() for cater in cater_list]
        ## operation or on all of binary_to_cater[cater_list]
        bin_mask = [binary_string_to_num(cater_to_bin.get(cater, '0'*13)) for cater in cater_list]
        result = 0
        for num in bin_mask:
            result = result | num
        mask_list.append(bin(result)[2:].zfill(13))
    return mask_list









In [49]:

conn = sqlite3.connect('result\placev2.db')
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM places")
total_places = cursor.fetchone()[0]
print(total_places)
cursor.execute("SELECT categories_0, categoryname, cuisine_tags, primary_cuisine from places")
categories = cursor.fetchall()
print(len(categories))
mask_list = process_catergories(categories)
mask_list = [list(binary_string) for binary_string in mask_list]
print(mask_list[:50])


917
917
[['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0

In [50]:
# cursor.execute("DROP COLUMN a if a.contains(cat_name)")
for cat in CATEGORIES:
    cat_name = f'cat_{cat}'
    cursor.execute(f"alter table places drop column {cat_name}")
    cursor.execute(f'ALTER TABLE places ADD COLUMN "{cat_name}" INTEGER DEFAULT 0')

conn.commit()



In [51]:
cursor.execute("SELECT rowid FROM places ORDER BY rowid ASC")
rows = cursor.fetchall()
ids = [r[0] for r in rows]

assert len(ids) == len(mask_list)
print(ids[:50])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [52]:
for i, cat in enumerate(CATEGORIES):
    cat_name = f'cat_{cat}'
    for j, place_id in enumerate(ids):
        bit = int(mask_list[j][i])  # '0'/'1' -> 0/1
        cursor.execute(
            f'UPDATE places SET "{cat_name}" = ? WHERE rowid = ?',
            (bit, place_id)
        )

conn.commit()

In [53]:
## analyze num of each cat
for cat in CATEGORIES:
    cat_name = f'cat_{cat}'
    cursor.execute(f'SELECT COUNT(*) FROM places WHERE {cat_name} = 1')
    num = cursor.fetchall()
    print(cat, num)

accommodation [(33,)]
activity [(36,)]
catering [(587,)]
commercial [(116,)]
entertainment [(85,)]
heritage [(56,)]
leisure [(132,)]
manufacturing [(22,)]
natural [(38,)]
religion [(12,)]
rental [(16,)]
service [(152,)]
tourism [(71,)]


In [54]:
cursor.execute("SELECT rowid, title FROM places WHERE LOWER(title) LIKE ?", ("%công viên%",))
result = cursor.fetchall()
print([mask_list[id] for id, title in result])

[['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1'], ['0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0']]


In [56]:
all_zeros = ' AND '.join([f'cat_{cat} = 0' for cat in CATEGORIES])
print(all_zeros)  # kiểm tra: cat_accommodation = 0 AND cat_activity = 0 AND ...

cursor.execute(f'SELECT title FROM places WHERE {all_zeros}')
result = cursor.fetchall()
print(result)
print(len(result))

cat_accommodation = 0 AND cat_activity = 0 AND cat_catering = 0 AND cat_commercial = 0 AND cat_entertainment = 0 AND cat_heritage = 0 AND cat_leisure = 0 AND cat_manufacturing = 0 AND cat_natural = 0 AND cat_religion = 0 AND cat_rental = 0 AND cat_service = 0 AND cat_tourism = 0
[('Supermarket Beauty Le Hong Phong',), ('Shophouse Heritage',), ("Kenny's Home",), ('Dai Tung Manufacturing & Trading Co., Ltd.',), ('Lux Massage Nuru & Lingam tại nhà - Sensual Outcall Massage',), ('Ashikata Spa',), ('Công Ty Tnhh Heritage Group',), ('Shophouse Heritage',), ('Công Ty Tnhh Heritage Group',), ('Công Ty Tnhh Td Heritage Hospitality',), ('KOHLER',), ('REHAU Vietnam',), ('TTI Vietnam Văn phòng quận 1',), ('CÔNG TY CỔ PHẦN LUCERO',), ('Hira Industries LLC - Vietnam Representative Office',), ('Vpđd Saemwoo Chemical Manufacturing Corp',), ('Optical Fiber Cable Manufacturing J.V C',), ('Sân khấu Sen Hồng',), ('CuChi Tunnels Tours',)]
19


In [15]:
import sqlite3

conn = sqlite3.connect('result\places.db')
cursor = conn.cursor()

## change binary to list of catergories
def binary_to_cater(binary_mask):
    return [CATEGORIES[i] for i in range(len(binary_mask)) if binary_mask[i] == '1']

list_cater = ','.join([f"cat_{cat}" for cat in CATEGORIES])

cursor.execute(f"select address, {list_cater} from places")
categories = cursor.fetchall()

all_cat = []
for item in categories:
    list_item = [str(i) for i in item]
    list_cat = binary_to_cater(list_item[1:])
    all_cat.append((list_item[0], '[' + ','.join(list_cat) + ']'))
    print(all_cat[-1])
## add all cat to column categories in places table
cursor.execute(f"ALTER TABLE places ADD COLUMN categories TEXT")
for i, cat in enumerate(all_cat):
    cursor.execute(f"UPDATE places SET categories = ? WHERE address = ?", (cat[1], cat[0]))

conn.commit()









('19 Đặng Dung, Phường Tân Định, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[commercial]')
('3/12 Đề Thám, Phường Cô Giang, Quận 1, Thành phố Hồ Chí Minh 70000, Vietnam', '[catering,commercial,service]')
('35 Ký Con, Phường Nguyễn Thái Bình, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('37A Cô Bắc, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh 70000, Vietnam', '[catering]')
('26 Cô Giang, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('16A Cô Giang, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('120B Bến Vân Đồn, Phường 9, Quận 4, Thành phố Hồ Chí Minh 700000, Vietnam', '[catering]')
('117 Võ Văn Kiệt, Phường Nguyễn Thái Bình, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('184 Võ Văn Kiệt, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('1a Nguyễn Hữu Hào, Phường 9, Quận 4, Thành phố Hồ Chí Minh, Vietnam', '[catering]')
('40 Đề Thám, Phường Cầu Ông Lãnh, Quận 1, Thành phố 

In [13]:
## remove column categories
cursor.execute("alter table places drop column categories")

conn.commit()




In [16]:

print(','.join([f"cat_{cater}" for cater in CATEGORIES]))

cat_accommodation,cat_activity,cat_catering,cat_commercial,cat_entertainment,cat_heritage,cat_leisure,cat_manufacturing,cat_natural,cat_religion,cat_rental,cat_service,cat_tourism
